In [ ]:
from keras.applications.resnet50 import ResNet50
import numpy as np
np.set_printoptions(threshold=np.nan)

from art.classifiers import KerasClassifier

# Load model
mean_imagenet = np.zeros([224, 224,3])
mean_imagenet[...,0].fill(103.939)
mean_imagenet[...,1].fill(116.779)
mean_imagenet[...,2].fill(123.68)
model = ResNet50(weights='imagenet')
classifier = KerasClassifier(model=model, clip_values=(0, 255),  preprocessing=(mean_imagenet,1))

In [ ]:
from os.path import join, abspath, expanduser

# Get Imagenet labels
path = expanduser('~/git/nemesis')
with open(join(path, "imagenet/labels.txt"), "r") as f_input:
    class_names = eval(f_input.read())

In [ ]:
from keras.preprocessing import image

path = expanduser('~/git/nemesis/test_api/clean100')
image_files = ['n04479046_2998_224x224.jpg']
images = []

for i in image_files:
    image_file = join(path, i)
    image_ = image.load_img(image_file, target_size=(224, 224))
    img = image.img_to_array(image_)
    images.append(img)
images = np.array(images)
    
# Original predictions
preds = np.argmax(classifier.predict(images), axis=1)
print('Predictions:')
for p in preds:
    print(class_names[p])

In [ ]:
levels = ['low', 'med', 'high']

In [ ]:
# Create attacks
from art.attacks import FastGradientMethod, SaliencyMapMethod, CarliniL2Method, DeepFool, NewtonFool
from art.attacks import ProjectedGradientDescent, BasicIterativeMethod, VirtualAdversarialMethod

attacks_config = {
    'fgsm': {'class': FastGradientMethod,
             'low': {'eps': 5},
             'med': {'eps': 15},
             'high': {'eps': 30}},
    'carlini': {'class': CarliniL2Method,
               'low': {'confidence': 0, 'targeted': False, 'binary_search_steps': 1, 'max_iter': 10},
               'med': {'confidence': 20, 'targeted': False, 'binary_search_steps': 5, 'max_iter': 30},
               'high': {'confidence': 50, 'targeted': False, 'binary_search_steps': 10, 'max_iter': 100}},
    'jsma': {'class': SaliencyMapMethod,
             'low': {'theta': 1, 'gamma': .1},
             'med': {'theta': 5, 'gamma': .3},
             'high': {'theta': 10, 'gamma': 1.}},
    'bim':  {'class': BasicIterativeMethod,
             'low': {'eps': 5, 'eps_step': 1, 'max_iter': 20},
             'med': {'eps': 20, 'eps_step': 1, 'max_iter': 50},
             'high': {'eps': 50, 'eps_step':1, 'max_iter': 100}},
    'pgd':  {'class': ProjectedGradientDescent,
             'low': {'eps': 10, 'eps_step': 1, 'max_iter': 20},
             'med': {'eps': 20, 'eps_step': 1, 'max_iter': 50},
             'high': {'eps': 50, 'eps_step':1, 'max_iter': 100}},
    'deepfool': {'class': DeepFool,
                'low': {'max_iter': 10},
                'med': {'max_iter': 30},
                'high': {'max_iter': 100}},
    'newtonfool': {'class': NewtonFool,
                  'low': {'max_iter': 10, 'eta': .01},
                  'med': {'max_iter': 30, 'eta': .1},
                  'high': {'max_iter': 100, 'eta': .3}}
}

In [ ]:
def get_attack(classifier, name, level='low'):
    try:
        attacker = attacks_config[name]['class'](classifier)
        attacker.set_params(**attacks_config[name][level])
        return attacker
    except:
        raise KeyError()

In [ ]:
# Create defenses
from art.defences import GaussianAugmentation, FeatureSqueezing, SpatialSmoothing

ga = GaussianAugmentation(sigma=5, augmentation=False)


defenses_config = {
    'ga': {'class': GaussianAugmentation,
          'low': {'sigma': 5, 'augmentation': False},
          'med': {'sigma': 10, 'augmentation': False},
          'high': {'sigma': 15, 'augmentation': False}},
    'fs': {'class': FeatureSqueezing,
          'low': {'bit_depth': 5},
          'med': {'bit_depth': 3},
          'high': {'bit_depth': 1}},
    'ss': {'class': SpatialSmoothing,
          'low': {'window_size': 3},
          'med': {'window_size': 9},
          'high': {'window_size': 15}}
}

In [ ]:
def get_defense(name, level='low'):
    try:
        defense = defenses_config[name]['class'](**defenses_config[name][level])
        return defense
    except:
        raise KeyError()

In [ ]:
# Compute and save all images
for i in images:
    for a in attacks_config.keys():
        # Run attack
        for la in levels:
            attack = get_attack(classifier, a, level=la)
            
            images_adv = attack.generate(images)
            # Save attack images here; one could also check predictions at this time to validate attack configs
            
            # Now add defenses to the attack
            for d in defenses_config.keys():
                for ld in levels:
                    defense = get_defense(d, level=ld)
                    images_adv_def = defense(images_adv)
                    # Save combination of attack and defense here
    
    # At this point, we have attack and attack + defense images
    # Now compute with defense alone
    for d in defenses_config.keys():
        for ld in levels:
            defense = get_defense(d, level=ld)
            images_def = defense(images)
            
            # Save defense images here